In [3]:
#import libs
import pandas as pd
import os
from torch.utils.data import Dataset, DataLoader
import cv2
from torchvision import transforms
from time import time
import numpy as np
from PIL import Image
import torch
from utils import CFG as utils

In [4]:

    
utils.seed_everything(42)


# Distribution of the target variable & Preprocces with DataFrame

In [10]:
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')

In [ ]:
train_df.info()
train_df.isna().sum()

In [ ]:
train_df.item_nm.describe()

In [13]:
assert(len(train_df) == len(os.listdir('./data/train')))

# DataModule

In [14]:
class ImageData(Dataset):
    '''
    Main DataModule
    '''
    def __init__(self,df ,test = True ,transform = None):
        self.df = df
        self.transform = transform
        self.test = test
        
        
        
        
    def __len__(self):
        return len(self.df)
    
    
    def __getitem__(self, index):
        path = ['./data/queries','./data/test/']
        img_path = path[self.test] + f'{self.df.idx.iloc[index]}' + '.png'
        img = Image.open(img_path)
        img = self.transform(img)
        if img.size(0) != 3:
            to_rgb = transforms.Lambda(lambda x: x[:3])
            img = to_rgb(img)
        
        return img
        

In [15]:
transform = transforms.Compose([transforms.Resize((128,128)),
                                transforms.ToTensor()])
data = ImageData(test_df,transform=transform)


In [17]:
dataloader = utils.loader(data,utils.batch_size)

In [18]:
next(iter(dataloader)).shape

# MODEL DCEC

![Image of Yaktocat](https://www.researchgate.net/profile/Xifeng-Guo/publication/320658590/figure/fig2/AS:614154637410315@1523437284434/The-structure-of-deep-convolutional-embedded-clustering-DCEC-It-is-composed-of-a.png)


In [80]:
class ClusterlingLayer(nn.Module):
    def __init__(self, in_features=10, out_features=10, alpha=1.0):
        super(ClusterlingLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.alpha = alpha
        self.weight = nn.Parameter(torch.Tensor(self.out_features, self.in_features))
        self.weight = nn.init.xavier_uniform_(self.weight)

    def forward(self, x):
        x = x.unsqueeze(1) - self.weight
        x = torch.mul(x, x)
        x = torch.sum(x)
        x = 1.0 + (x / self.alpha)
        x = 1.0 / x
        x = x ** ((self.alpha +1.0) / 2.0)
        x = torch.t(x) / torch.sum(x)
        x = torch.t(x)
        return x

In [202]:
class DCEC(nn.Module):

    def __init__(self, input_size=[3, 128, 128], channels=[32, 64, 128],
                 num_clusters=10, activation_out = False):
        super(DCEC, self).__init__()
        self.params = input_size
        self.channels = channels
        self.num_clusters = num_clusters
        self.cluster = ClusterlingLayer()
        self.relu = nn.ReLU(inplace=False)
        #ENCODER
        self.encoder = nn.Sequential(
            nn.Conv2d(self.params[0], self.channels[0], 5, stride=2,
                       padding=2),
            nn.ReLU(inplace=False),
            nn.Conv2d(self.channels[0], self.channels[1], 5, stride=2,
                      padding=2),
            nn.ReLU(inplace=False),
            nn.Conv2d(self.channels[1], self.channels[2], 3, stride=2,
                      padding=0)
        )
        
        
        # EMBED
        self.lin_features_len = ((self.params[1] // 2 // 2 - 1) // 2) * (
                    (self.params[1] // 2 // 2 - 1) // 2) * channels[2]
        self.embedding = nn.Linear(self.lin_features_len, self.num_clusters)
        self.deembedding = nn.Linear(self.num_clusters, self.lin_features_len)

        # DECODER
        out_pad = 1 if self.params[1] // 2 // 2 % 2 == 0 else 0
        self.deconv3 = nn.ConvTranspose2d(self.channels[2], self.channels[1],
                                          3, stride=2, padding=0,
                                          output_padding=out_pad)
        out_pad = 1 if self.params[1] // 2 % 2 == 0 else 0
        self.deconv2 = nn.ConvTranspose2d(self.channels[1], self.channels[0],
                                          5, stride=2, padding=2,
                                          output_padding=out_pad)
        out_pad = 1 if self.params[1] % 2 == 0 else 0
        self.deconv1 = nn.ConvTranspose2d(self.channels[0], self.params[0], 5,
                                          stride=2, padding=2,
                                          output_padding=out_pad)

        # LOSS_FUNC
        self.relu1_1 = self.relu
        self.relu2_1 = self.relu
        self.relu3_1 = self.relu
        self.relu1_2 = self.relu
        self.relu2_2 = self.relu
        self.relu3_2 = self.relu
        self.tanh = nn.Tanh()

    def forward(self, x):
        # ENCODE
        
        x = self.encoder(x)
        # matrix to array & clustering_out
        x = x.view(x.size(0), -1)
        x = self.embedding(x)
        
        clustering_out = self.cluster(x)
        
        # decode
        x = self.deembedding(x)
        
        x = self.relu1_2(x)
        x = x.reshape(x.shape[0], self.channels[-1],
                      (self.params[1] // 2 // 2 - 1) // 2,
                      (self.params[1] // 2 // 2 - 1) // 2)
        x = self.deconv3(x)
        x = self.relu2_2(x)
        
        x = self.deconv2(x)
        x = self.relu3_2(x)
        
        x = self.deconv1(x)
        return x, clustering_out


In [ ]:
.